# BaT DATA PULL

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import joblib
import os
import sys
import helpers
from glob import glob
from google.cloud import bigquery
import difflib


Get List of Every Result and Save:

In [3]:
# Get List of Every Result and Save
make = input("Enter Car Make: ")
model = input("Enter Car Model: ")
delimiter = "%newline%"
data = ""


open("./data/HTML/" + make + model + ".html", "w").close()

if model == "":
    listings = helpers.get_listings_no_model(make)
    souplist = joblib.Parallel(n_jobs=-2)(
        joblib.delayed(helpers.get_data)(url) for url in listings
    )

    temp = delimiter.join(souplist)
    data += delimiter + temp
else:
    ids, urls = helpers.get_listings(make, model)
    listings = []

    # Iterate through BaT specific Id's and show more pages for each listing
    urlend = "&results=items"
    pgend = 15
    for i in range(0, len(ids)):
        urlstart = (
            "https://bringatrailer.com/wp-json/bringatrailer/1.0/data/keyword-filter?bat_keyword_pages="
            + ids[i]
            + "&sort=td&page="
        )
        tempurl = helpers.get_urls(urlstart, urlend, pgend, urls)
        listings = tempurl
        listings = listings[len(ids) : len(listings)]
        souplist = joblib.Parallel(n_jobs=-2)(
            joblib.delayed(helpers.get_data)(url) for url in listings
        )

        temp = delimiter.join(souplist)
        data += delimiter + temp

# Write data to file
with open("./data/HTML/" + make + model + ".html", "a") as file:
    file.write(data)

display("done")


'done'

Pull Information Off Each Listing Stored in URLS List:

In [4]:
carlist = []
Car = []
### Read Data and Save to DataFrame ###

with open("./data/HTML/" + make + model + ".html", "r") as file:
    test_data = file.read()

out = test_data.split(delimiter)

# Iterate through each listings and test again to filter out non-car listings using title this time
for i in range(0, len(listings)):
    soup = BeautifulSoup(out[i], "html.parser")
    title = str(soup.find("title"))
    if (
        "Parts" not in title
        and "Tool" not in title
        and "Memorabilia" not in title
        and "Luggage" not in title
        and "Tool Kit" not in title
        and "Removable" not in title
        and "Gearbox" not in title
        and title != "None"
        and "listing" in listings[i]
    ):
        loc = str(soup.findAll("a"))
        transmission = str(soup.findAll("li"))
        contents = str(soup.findAll("p"))
        essentials = soup.find("div", class_="essentials")

        # Get Car Description
        desc = helpers.get_desc(listings[i])

        # Get Year
        year = desc[0:4]

        # Check if Sold and get highest bid
        sold, price = helpers.get_sale_price(title, loc)

        # Get Transmission
        trans = "Manual"
        if "Automatic" in transmission:
            trans = "Automatic"

        # Get Location
        town, state = helpers.get_location(loc)

        # Check for Project
        if "project" in title or "Project" in title:
            project = 1
        else:
            project = 0

        # Get Lot Number
        try:
            index = title.index("Lot #")
            lotnumb = title[index + 5 : index + 12].partition(")")[0]
        except:
            lotnumb = "N/A"

        # Check for Replica
        if "replica" in title or "Replica" in title:
            replica = 1
        else:
            replica = 0

        # Get Month
        month = helpers.get_month(title)

        # Get Year
        if "(" in title:
            index = title.index("(")
            yearsold = title[index - 5 : index - 1]
        else:
            index = title.index("20")
            yearsold = title[index : index + 5]

        # Get Engine and Number Matching
        numbmatch = 0
        if "Numbers-Matching" in transmission:
            numbmatch = 1
        engine = helpers.get_engine(transmission)

        # Get Various Indicators for rust, refurbishment, restoration, scratch, paint bubbles, metal repair, hardtop, or overdrive options
        (
            rust,
            refurbished,
            restored,
            scratch,
            paintbub,
            metalrepair,
            hardtop,
            overdrive,
            turbo,
            super,
        ) = helpers.get_indicators(contents)

        # Get Mileage
        miles, milestmu = helpers.get_mileage(transmission)

        # Get Descriptions of options and chassis #
        (
            chassis,
            specialdesc,
            mileagedesc,
            enginedesc,
            transdesc,
            paintdesc,
            interiordesc,
            carbdesc,
            wheeldesc,
            brakedesc,
            suspdesc,
            engine,
        ) = helpers.get_engine_desc(essentials, engine)

        # Check Trans with Description
        if "Manual" in transdesc:
            trans = "Manual"
        elif "Automatic" in transdesc:
            trans = "Automatic"

        # Check Mileage with Description (TODO: remove try catch)
        try:
            if "K" in str(miles):
                index = mileagedesc.index("ilometers")
                tempmile = mileagedesc[0:index].partition(" ")[0]
                if "k" in tempmile or "K" in tempmile:
                    milestmu = re.sub(r"[^0-9]", "", tempmile)
                    milestmu = int(milestmu) * 1000
            elif "K" in str(milestmu):
                index = mileagedesc.index("ilometers")
                tempmile = mileagedesc[0:index].partition(" ")[0]
                if "k" in tempmile or "K" in tempmile:
                    miles = re.sub(r"[^0-9]", "", tempmile)
                    miles = int(miles) * 1000
        except:
            print("failed")

        # Check for bespoke bodies
        if "Bertone" in title:
            bertone = 1
        else:
            bertone = 0
        if "Zagato" in title:
            zagato = 1
        else:
            zagato = 0

        Car = [
            lotnumb,
            desc,
            trans,
            year,
            miles,
            milestmu,
            engine,
            turbo,
            super,
            numbmatch,
            hardtop,
            overdrive,
            scratch,
            paintbub,
            metalrepair,
            replica,
            rust,
            refurbished,
            restored,
            project,
            sold,
            month,
            yearsold,
            price,
            town,
            state,
            zagato,
            bertone,
            chassis,
            specialdesc,
            mileagedesc,
            enginedesc,
            transdesc,
            paintdesc,
            interiordesc,
            carbdesc,
            wheeldesc,
            brakedesc,
            suspdesc,
            listings[i],
            make,
            model,
        ]

        carlist.append(Car)

    else:
        print("Discarded")

# Export to DataFrame
df = pd.DataFrame(
    carlist,
    columns=[
        "Lot Number",
        "Description",
        "Transmission",
        "Model Year",
        "Mileage",
        "Mileage (if TMU)",
        "Engine",
        "Turbocharged",
        "Supercharged",
        "Numbers Matching",
        "Hardtop",
        "Overdrive",
        "Scratch",
        "Paint Bubble",
        "Metal Repair",
        "Replica",
        "Rust",
        "Refurbished",
        "Restored",
        "Project",
        "Sold",
        "Month",
        "Year",
        "Price",
        "City",
        "State",
        "Zagato",
        "Bertone",
        "Chassis",
        "Special Description",
        "Mileage Description",
        "Engine Description",
        "Transmission Description",
        "Paint Description",
        "Interior Description",
        "Carburetor Description",
        "Wheels Description",
        "Brakes Description",
        "Suspension Description",
        "URL",
        "Make",
        "Model",
    ],
)

display(df)


Discarded


,Lot Number,Description,Transmission,Model Year,Mileage,Mileage (if TMU),Engine,Turbocharged,Supercharged,Numbers Matching,...,Transmission Description,Paint Description,Interior Description,Carburetor Description,Wheels Description,Brakes Description,Suspension Description,URL,Make,Model
0,"93,333",1968 mg mgc,Manual,1968,85000,N/A,2.9,0,0,0,...,4-Speed Manual Transmission w/Overdrive,"Silver-Painted 15"" Wire Wheels",Black Upholstery w/White Piping,N/A,"Silver-Painted 15"" Wire Wheels",Front Disc Brakes,N/A,https://bringatrailer.com/listing/1968-mg-mgc-3/,MG,MGC
1,"92,874",1969 mg mgc,Automatic,1969,TMU,52000,2.9,0,0,0,...,Three-Speed Automatic Transmission,Repainted Red,Black Vinyl Upholstery,Twin SU Carburetors,"Chrome-Finished 15"" Wire Wheels",Front Disc Brakes,N/A,https://bringatrailer.com/listing/1969-mg-mgc-15/,MG,MGC
2,"81,087",1969 mg mgc gt,Manual,1969,54000,N/A,2.9,0,0,0,...,4-Speed Manual Transmission w/OD,"Silver-Painted 15"" Wire Wheels",Black Leather Upholstery,Dual SU Carburetors,"Silver-Painted 15"" Wire Wheels",Uprated Front Disc Brakes,N/A,https://bringatrailer.com/listing/1969-mg-mgc-...,MG,MGC
3,"80,200",1969 mg mgc,Manual,1969,TMU,77000,2.9,0,0,0,...,Four-Speed Manual Transmission w/Overdrive,Green Paint,Black Upholstery,N/A,"15"" Wire Wheels",N/A,N/A,https://bringatrailer.com/listing/1969-mg-mgc-14/,MG,MGC
4,"75,736",1968 mg mgc gt,Manual,1968,71000,N/A,2.9,0,0,0,...,4-Speed Manual Transmission w/OD,Red Paint,Checker Plate Interior Panels,N/A,"Chrome-Finished 15"" Wire Wheels",Power-Assisted Front Disc Brakes,N/A,https://bringatrailer.com/listing/1968-mg-mgc-...,MG,MGC
5,"73,435",1969 mg mgc,Manual,1969,TMU,14000,2.9,0,0,0,...,Four-Speed Manual Transmission w/Overdrive,Yellow Paint,LED Instrumentation & Interior Lighting,Triple SU Carburetors,"15"" Wire Wheels w/Ear-Less Spinners",Upgraded Power-Assisted Front Disc Brakes,Polyurethane Suspension Bushings,https://bringatrailer.com/listing/1969-mg-mgc-11/,MG,MGC
6,"59,686",1968 mg mgb gt,Manual,1968,TMU,14000,2.9,0,0,0,...,4-Speed Manual Gearbox w/ Overdrive,Repainted Green in 2005,Black Leather Interior,N/A,N/A,N/A,N/A,https://bringatrailer.com/listing/1968-mg-mgb-...,MG,MGC
7,"59,588",1968 mg mgc gt,Manual,1968,TMU,2500,2.9,0,0,0,...,Four-Speed Manual Transmission w/OD,N/A,Black Upholstery w/White Piping,N/A,"15"" Wire Wheels",N/A,N/A,https://bringatrailer.com/listing/1968-mg-mgc-...,MG,MGC
8,"54,015",1969 mg mgc,Manual,1969,TMU - K,1300000,3.0,0,0,0,...,4-Speed Manual Transmission w/Overdrive,Repainted Green w/Yellow Accent,N/A,"2"" SU Carburetors","15"" Minilite Wheels",N/A,N/A,https://bringatrailer.com/listing/1969-mg-mgc-13/,MG,MGC
9,"52,805",1969 mg mgc gt,Manual,1969,58000,N/A,2.9,0,0,0,...,Four-Speed Manual Transmission W/Overdrive,Repainted British Racing Green,Black Leather Upholstery,Triple Weber DCOE Carburetors,"15"" Dayton Wire Wheels",Modified Brakes,Upgraded Suspension,https://bringatrailer.com/listing/1969-mg-mgc-...,MG,MGC


Save Data as Excel File:

In [88]:
path = "data/" + make + "/"

try:
    os.mkdir(path)
except:
    print("Path already exists")

df.to_excel(path + make + "_" + model + ".xlsx")

print("Saved!")


Saved!


# AGGREGATE AND CLEAN DATA

In [6]:
#Aggregate all the excel files into one DataFrame
fnames = [y for x in os.walk("./data") for y in glob(os.path.join(x[0], "*.xlsx"))]
df = pd.concat(
    joblib.Parallel(n_jobs=-2)(
        joblib.delayed(pd.read_excel)(fname) for fname in fnames
    ),
    axis=0,
)


In [42]:
# Import Partially Cleaned Data TODO Clean data more thoroughly
df = pd.read_excel('data/final_data/final_cleaned_data.xlsx')

In [44]:
# Build Schema for Database
schema = [
    bigquery.SchemaField(
        name="turbocharged",
        field_type=bigquery.enums.SqlTypeNames.INTEGER,
        description="0 if not turbocharged, 1 if turbocharged."
    ),
    bigquery.SchemaField(
    name="supercharged",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="0 if not supercharged, 1 if supercharged."
    ),
    bigquery.SchemaField(
    name="lot_number",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="Unique identifier for BaT auction listings"
    ),
    bigquery.SchemaField(
    name="description",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="Description of car, pulled from auction title"
    ),
    bigquery.SchemaField(
    name="transmission",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="Automatic or Manual Transmission"
    ),
    bigquery.SchemaField(
    name="model_year",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="Year the car was built"
    ),
    bigquery.SchemaField(
    name="mileage",
    field_type=bigquery.enums.SqlTypeNames.FLOAT,
    description="Mileage on the car if known"
    ),
    bigquery.SchemaField(
    name="mileage_if_tmu",
    field_type=bigquery.enums.SqlTypeNames.FLOAT,
    description="Mileage on the car if not fully known"
    ),
    bigquery.SchemaField(
    name="engine",
    field_type=bigquery.enums.SqlTypeNames.FLOAT,
    description="Engine displacement in liters"
    ),
    bigquery.SchemaField(
    name="numbers_matching",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="1 if the term 'Numbers Matching' is found in contents, 0 if not"
    ),
    bigquery.SchemaField(
    name="hardtop",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="1 if 'hardtop' found in contents, 0 if not"
    ),
    bigquery.SchemaField(
    name="overdrive",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="1 if 'overdrive' found in contents, 0 if not"
    ),
    bigquery.SchemaField(
    name="scratch",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="1 if 'scratch' found in contents, 0 if not"
    ),
    bigquery.SchemaField(
    name="paint_bubble",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="1 if 'paint bubble' found in contents, 0 if not"
    ),
    bigquery.SchemaField(
    name="metal_repair",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="1 if 'metal_repair' found in contents, 0 if not"
    ),
    bigquery.SchemaField(
    name="rust",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="1 if 'rust' found in contents, 0 if not"
    ),
    bigquery.SchemaField(
    name="refurbished",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="1 if 'refurbished' found in contents, 0 if not"
    ),
    bigquery.SchemaField(
    name="restored",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="1 if 'restored' found in contents, 0 if not"
    ),
    bigquery.SchemaField(
    name="replica",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="1 if 'replica' found in contents, 0 if not"
    ),
    bigquery.SchemaField(
    name="project",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="1 if 'project' found in contents, 0 if not"
    ),
    bigquery.SchemaField(
    name="sold",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="1 if sold found in contents, 0 if did not meet reserve"
    ),
    bigquery.SchemaField(
    name="month",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="Numerical month that car was offered"
    ),
    bigquery.SchemaField(
    name="year",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="Year that car was offered"
    ),
    bigquery.SchemaField(
    name="price",
    field_type=bigquery.enums.SqlTypeNames.FLOAT,
    description="Price car sold for or was bid to"
    ),
    bigquery.SchemaField(
    name="city",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="City where car was offered from"
    ),
    bigquery.SchemaField(
    name="state",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="State where car was offered from"
    ),
    bigquery.SchemaField(
    name="zagato",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="1 if 'Zagato' in title, 0 if not"
    ),
    bigquery.SchemaField(
    name="bertone",
    field_type=bigquery.enums.SqlTypeNames.INTEGER,
    description="1 if 'Bertone' in title, 0 if not"
    ),
    bigquery.SchemaField(
    name="chassis",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="Chassis Number (sometimes engine number listed instead)"
    ),
    bigquery.SchemaField(
    name="special_description",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="Special Description Attributes"
    ),
    bigquery.SchemaField(
    name="engine_description",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="Description of engine displacement and layour"
    ),
    bigquery.SchemaField(
    name="mileage_description",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="Description of mileage"
    ),
    bigquery.SchemaField(
    name="transmission_description",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="Description of transmission layout"
    ),
    bigquery.SchemaField(
    name="paint_description",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="Description of paint color"
    ),
    bigquery.SchemaField(
    name="interior_description",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="Description of interior"
    ),
    bigquery.SchemaField(
    name="carburetor_description",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="Description of carburetors (if applicable)"
    ),
    bigquery.SchemaField(
    name="wheels_description",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="Description of wheels"
    ),
    bigquery.SchemaField(
    name="brakes_description",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="Description of brakes"
    ),
    bigquery.SchemaField(
    name="suspension_description",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="Description of suspension"
    ),
    bigquery.SchemaField(
    name="url",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="Listing url"
    ),
    bigquery.SchemaField(
    name="make",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="Make name"
    ),
    bigquery.SchemaField(
    name="model",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="Model name (likely not correct)"
    ),
    bigquery.SchemaField(
    name="kilometer",
    field_type=bigquery.enums.SqlTypeNames.STRING,
    description="'K' if mileage is in kilometers, blank if not"
    ),
]



In [45]:
#Load DataFrame into Google Cloud
project_id = "earnest-episode-372201"
client = bigquery.Client(project=project_id)
job_config = bigquery.LoadJobConfig(
    destination_table_description="Table of vintage car sales data. One row per car sale.",
    write_disposition="WRITE_APPEND",
    schema = schema
)


client.load_table_from_dataframe(
    dataframe=df[[
        "lot_number", 
        "description", 
        "transmission", 
        "model_year",
        "mileage",
        "mileage_if_tmu", 
        "kilometer", 
        "engine", 
        "turbocharged", 
        "supercharged", 
        "numbers_matching",
        "hardtop",
        "overdrive",
        "scratch",
        "paint_bubble",
        "metal_repair",
        "replica",
        "rust",
        "refurbished",
        "restored",
        "project",
        "sold",
        "month",
        "year",
        "price",
        "city",
        "state",
        "zagato",
        "bertone",
        "chassis",
        "special_description",
        "mileage_description",
        "engine_description",
        "transmission_description",
        "paint_description",
        "interior_description",
        "carburetor_description",
        "wheels_description",
        "brakes_description",
        "suspension_description",
        "url",
        "make",
        "model",
        ]],
    destination=f"{project_id}.cars.vintage_car_updated",
    job_config=job_config,
)


LoadJob<project=earnest-episode-372201, location=US, id=c57530d8-ee15-43c7-b44b-a7b69040d502>